# Importing Libraries and Downloading Dataset

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import keras
import os
import numpy as np

vocab_size = 88584

max_len = 250
batch_size = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = vocab_size)

# Padding the Data

In [ ]:
train_data = sequence.pad_sequences(train_data,max_len)
test_data = sequence.pad_sequences(test_data, max_len)

# Creating Model

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,32),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])

# Lookin' what we created

In [ ]:
model.summary()

# Training Model

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop",metrics=["acc"])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

# Results (Testing) 

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

# Making Predictions

In [ ]:
word_index = imdb.get_word_index()

def text_encoder(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],max_len)[0]

text = "That movie was just amazing, so amazing"
encoded = text_encoder(text)
print(encoded)

In [ ]:
reverse_word_index = {value: key for (key,value) in word_index.items()}

def integer_decoder(integers):
  pad = 0
  text = ""
  for num in integers:
    if num != pad:
      text += reverse_word_index[num] + " "
  return text[:-1]

print(integer_decoder(encoded))

In [ ]:
def predict(text):
  encoded_text = text_encoder(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so nice! I really liked it and would watch it again because it was amazingly great!"
predict(positive_review)

negative_review = "That movie was horrible. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

In [ ]:
# Values above give us the percentage of positiveness in the statement.
# In the training section, it's seen model overfits depending on the small size of dataset. 
# However the results and predictions are not that bad, but of course thye can be improved.